In [15]:
from IPython.display import clear_output
import math


class figure:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def check(self, px, py, x, y, n):
        global game
        #print(x, y, n)                             #Вывод для промежутончой проверки
        for i in range (1, n + 1):
            #print('!!!!!!!!!!!!!!!!!!', px + i * x, py + i * y, ':')    #Вывод для промежутончой проверки
            if game.desk[px + i * x][py + i * y] == '.':
                continue
            else:
                return False
        return True
            
        
class pawn(figure):   #Класс Пешки
    def __init__(self, x, y, z):
        super().__init__(x, y)
        self.z = z
    def check(self, x, y, eat):
        if self.z == 0:
            if y == self.y:
                if self.x == 6 and x == 4 or abs(x - self.x) == 1:
                    if eat != '.':
                        return False
                    else:
                        return True
            else:
                if eat != '.' and eat.islower() == True:
                    return True
                return False
        else:
            if y == self.y:
                if self.x == 1 and x == 3 or abs(self.x - x) == 1:
                    if eat != '.':
                        return False
                    else:
                        return True
            else:
                if eat != '.' and eat.isupper() == True:
                    return True
                return False
            
    
class rook(figure):  #Класс Ладьи
    def __init__(self, x, y):
        super().__init__(x, y)
    def check(self, x, y):
        if x - self.x == 0:
            a = 1 if y > self.y else -1
            return(super().check(self.x, self.y, 0, a, abs(y - self.y)))
        elif y - self.y == 0:
            a = 1 if x > self.x else -1
            return(super().check(self.x, self.y, a, 0, abs(x - self.x)))
        else:
            return False

class knight(figure):  #Класс Коня
    def __init__(self, x, y):
        super().__init__(x, y)
    def check(self, x, y):
        if abs(x - self.x) == 2 and abs(y - self.y) == 1 or abs(x - self.x) == 1 and abs(y - self.y) == 2:
            return True
        else:
            return False
    
class queen(figure):    #Класс Ферзя
    def __init__(self, x, y):
        super().__init__(x, y)
    def check(self, x, y):
        #print(self.x, self.y, x, y, 'move')   #Вывод для промежутончой проверки
        if x - self.x == 0:
            a = 1 if y > self.y else -1
            return(super().check(self.x, self.y, 0, a, abs(y - self.y)))
        elif y - self.y == 0:
            a = 1 if x > self.x else -1
            return(super().check(self.x, self.y, a, 0, abs(x - self.x)))
        elif abs(x - self.x) == abs(y - self.y):
            a = 1 if x > self.x else -1
            b = 1 if y > self.y else -1
            return(super().check(self.x, self.y, a, b, abs(x - self.x)))#fe
        else:
            return False
    
class bishop(figure):   #Класс Слона
    def __init__(self, x, y):   
        super().__init__(x, y)
    def check(self, x, y):
        if abs(x - self.x) == abs(y - self.y):
            a = 1 if y > self.y else -1
            b = 1 if x > self.x else -1
            return(super().check(self.x, self.y, a, b, abs(x - self.x)))
        else:
            return False

class king(figure):    #Класс Короля
    def __init__(self, x, y):
        super().__init__(x, y)
    def check(self, x, y):
        if abs(x - self.x) <= 1 and abs(y - self.y) <= 1 and self.is_atacked(x, y) == False:
            return True
        else:
            return False


    
    
class game_board:
    cur = 0
    def __init__(self):
        cur = 0
        self.desk = [['.'] * 8]           #Инициализация(для доски)
        for i in range(0, 7):
            self.desk.append(['.'] * 8)
        self.desk[0][0], self.desk[0][7], self.desk[7][0], self.desk[7][7] = 'r', 'r', 'R', 'R'
        self.desk[0][1], self.desk[0][6], self.desk[7][1], self.desk[7][6] = 'h', 'h', 'H', 'H'
        self.desk[0][2], self.desk[0][5], self.desk[7][2], self.desk[7][5] = 'b', 'b', 'B', 'B'
        self.desk[0][3], self.desk[7][3] = 'q', 'Q'
        self.desk[0][4], self.desk[7][4] = 'k', 'K'
        self.desk[1][0:8] = 'p' * 8
        self.desk[6][0:8] = 'P' * 8
        self.show()

        
        
    def make_move(self, move):
        typ = move[0]
        global check_error
        check_error = 0
        
        if self.cur == 0:
            typ = typ.upper()
        else:
            typ = typ.lower()
        x, y, px, py = 0, 0, 0, 0
        self.eat = '.'
        det = 0
        move = move[1:]
        if move[0] == ':':            ###РЕализация съедания
            move = move[1:] + ':'
            
        if len(move) < 4:
            x = 8 - int(move[1])
            y = ord(move[0]) - ord('a')
        else:
            px = 8 - int(move[1])
            py = ord(move[0]) - ord('a')
            x = 8 - int(move[3])
            y = ord(move[2]) - ord('a')
            det = 1
        if move[len(move) - 1] == ':':
            self.eat = self.desk[x][y]
            self.desk[x][y] = '.'
        #Проверка правильности координат
        if x > 7 or y > 7 or x < 0 or y < 0 or px > 7 or px < 0 or py > 7 or py < 0:
            self.desk[x][y] = str(self.eat)
            return (self.error(8))
        

            
        if det == 1: # Если фигура задана однозначно ходом игрока
            if self.destination(typ.lower(), x, y, px, py) == True:
                self.desk[x][y] = self.desk[px][py]
                self.desk[px][py] = '.'
            else:
                self.desk[x][y] = str(self.eat)
                return (self.error(0)) # Подумат, как реализовать ошибки для разных фигур
        else: #Находим фигуры, которыми может быть сделан ход
            ver = self.find_f(typ)
            for i in range(0, len(ver), 2):
                px = ver[i]
                py = ver[i + 1]
                if self.destination(typ.lower(), x, y, px, py) == True:
                    self.desk[x][y] = self.desk[px][py]
                    self.desk[px][py] = '.'
                    break
            else:
                self.desk[x][y] = str(self.eat)
                return (self.error(0))
                
        
        #if self.cur == 0 and self.is_game_lost(bking) == True:
        #    print('Чёрным шах. Победа белых')
        #    return (-1)
        #elif self.cur == 1 and self.is_game_lost(wking) == True:
        #    print('Белым шах. Победа чёрных')
        #    return (-1)
        
        
        if check_error == 0:
            self.cur = (self.cur + 1) % 2
            clear_output(wait = True) 
            self.show()
        return (0)
                   
    #Распределение по классам
    def destination(self, typ, x, y, px, py):
        if typ == 'r':
            return(rook(px, py).check(x, y))
        if typ == 'h':
            return(knight(px, py).check(x, y))
        if typ == 'b':
            return(bishop(px, py).check(x, y))
        if typ == 'q':
            return(queen(px, py).check(x, y))
        if typ == 'k':
            return(king(px, py).check(x, y))
        if typ == 'p':
            return(pawn(px, py, self.cur).check(x, y, self.eat))
            
    #Нахождение фигур, которые могут сделать ход       
    def find_f(self, typ):
        ans = []
        for i in range(0, 8):
            for j in range(0, 8):
                if self.desk[i][j] == typ:
                    ans.append(i)
                    ans.append(j)
        return(ans)
                   
    #Вывод доски
    def show(self):                            
        print("    A B C D E F G H\n")
        j = 8
        for i in self.desk:
            print(j, " ", *i)
            j -= 1
        print("\nЧтобы закончить игру, введите: end")
                   
    #Проверка мата 
    def is_game_lost(self, king): 
        return False ## Пока не работает, костыль
        x = king.x
        y = king.y
        if self.is_atacked(x, y) == False:
            return False
        if x < 7 and self.is_atacked(x + 1, y) == False:
            return False
        if y < 7 and self.is_atacked(x, y + 1) == False:
            return False
        if x < 7 and y < 7 and self.is_atacked(x + 1, y + 1) == False:
            return False
        if x > 0 and self.is_atacked(x - 1, y) == False:
            return False
        if y > 0 and self.is_atacked(x, y - 1) == False:
            return False
        if x > 0 and y > 0 and self.is_atacked(x - 1, y - 1) == False:
            return False
        if x < 7 and y > 0 and self.is_atacked(x + 1, y - 1) == False:
            return False
        if x > 0 and y < 7 and self.is_atacked(x - 1, y + 1) == False:
            return False
        return True
        
    #Проверка, находится ли клетка под боем
    def is_atacked(self, x, y):
        cx = x + 1
        cy = y
        while cx < 8:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            ch1 = self.desk[cx][cy].upper()
            if self.desk[cx][cy] != '.' and t != self.cur:
                ch1 = self.desk[cx][cy].upper()
                if ch1 == 'Q' or ch1 == 'R' or ch1 == 'K' and cx == x + 1: 
                    return True
            elif t == self.cur:
                break
            cx += 1
        cx = x
        cy = y + 1
        while cy < 8:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                ch1 = self.desk[cx][cy].upper()
                if ch1 == 'Q' or ch1 == 'R' or ch1 == 'K' and cy == y + 1: 
                    return True
            elif t == self.cur:
                break
            cy += 1
        cx = x + 1
        cy = y + 1
        while cy < 8 and cx < 8:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                ch1 = self.desk[cx][cy].upper()
                if ch1 == 'Q' or ch1 == 'B' or (ch1 == 'K'or self.cur == 1 and ch1 == 'P') and cx == x + 1:
                    return True
            elif t == self.cur:
                break
            cy += 1
            cx += 1
        cx = x
        cy = y - 1
        while cy >= 0:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                if ch1 == 'Q' or ch1 == 'B' or ch1 == 'K' and cy == x - 1: 
                    return True
            elif t == self.cur:
                break
            cy -= 1
        cx = x - 1
        cy = y
        while cx >= 0:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                if ch1 == 'Q' or ch1 == 'B' or ch1 == 'K' and cx == x - 1:
                    return True
            elif t == self.cur:
                break
            cx -= 1
        cx = x + 1
        cy = y - 1
        while cy >= 0 and cx < 8:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                if ch1 == 'Q' or ch1 == 'B' or (ch1 == 'K'or self.cur == 1 and ch1 == 'P') and cx == x + 1:
                    return True
            elif t == self.cur:
                break
            cx += 1
            cy -= 1
        cx = x - 1
        cy = y + 1
        while cy < 8 and cx >= 0:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                if ch1 == 'Q' or ch1 == 'B' or (ch1 == 'K'or self.cur == 0 and ch1 == 'P') and cx == x - 1:
                    return True
            elif t == self.cur:
                break
            cx -= 1
            cy += 1
        cx = x - 1
        cy = y - 1
        while cy >= 0 and cx >= 0:
            t = (0 if self.desk[cx][cy].isupper() else 1)
            if self.desk[cx][cy] != '.' and t != self.cur:
                if ch1 == 'Q' or ch1 == 'B' or (ch1 == 'K'or self.cur == 0 and ch1 == 'P') and cx == x - 1:
                    return True
            elif t == self.cur:
                break
            cx -= 1
            cy -= 1
        return False
     #Сообщение об ошибке
    def error(self, typ):
        if typ == 0:
            print('Нарушены правила, по которым ходят фигуры')
            print('Напоминаю:')
            for i in range(1, 8):
                self.error(i)
        if typ == 2:
            print('Ладья ходит только по горизонтали или по вертикали')
        if typ == 4:
            print('Слон ходит только по диагонали')
        if typ == 3:
            print('Конь ходит только "буквой Г"')
        if typ == 5:
            print('Ферзь может ходить только по диагонали, вертикали или горизонтали на неограниченное число клеток(но никак по-другому)')
        if typ == 1:
            print('Пешка ходит только по направлению к фигурам противника на одну клетку по вертикали(на 2, если не двигалась раньше), и есть фигуры по диагонали впереди себя')
        if typ == 6:
            print('Король ходит только на одну клетку вокруг себя')
        if typ == 7:
            print('Ход фигуры не может перешагивать через другие')
        if typ == 8:
            print('Ход выходит за пределы шахматной доски')
        if typ == 9:
            print('Сейчас ход другого игрока')
        if typ == 10:
            print('Фигуры, чтобы сделать указанный ход не существует')
        ##Доделать все остальные виды ошибок, привязать их к вызовам функции
        #print("Неверный формат хода!")
        global check_error
        check_error = 1
        return 0
        
################################################


game = game_board()
bking = king(0, 4)
wking = king(7, 4)
king = wking
tmp = 0
check_error = 0

while tmp == 0:
    print('\nХод белых:' if game.cur == 0 else'\nХод чёрных:', end = ' ')
    a = input()    
    if a == 'end':
        break
    else:
        tmp = game.make_move(a)
    king = (wking if game.cur == 0 else bking)
print('end')

    A B C D E F G H

8   r h b q k b h r
7   p p . p p p p p
6   . . . . . . . .
5   . . p . . . . .
4   . . . . . . . .
3   . . . . P . . .
2   P P P P . P P P
1   R H B Q K B H R

Чтобы закончить игру, введите: end

Ход белых: Ke2


TypeError: 'king' object is not callable